In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras as K
from sklearn.base import BaseEstimator
import gc
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import euclidean, cosine

%matplotlib inline  

Using TensorFlow backend.


### Define Constants

In [20]:
BATCH_SIZE = 1
VEC_SIMILARITY = 10

### Load Data

In [28]:
with open('resources/rusvectors/model.txt') as f:
    lines = f.readlines()
    embeddings = []
    word_to_index = {}
    index_to_word = {}
    for index, line in enumerate(lines[1:]):
        values = line.split(' ')
        word = values[0].split('_')[0]
        embedding = np.array(values[1:]).astype('float64')
        embeddings.append(embedding)
        word_to_index[word] = index
        index_to_word[index] = word
    embeddings = np.asarray(embeddings)
    

In [29]:
d = pd.DataFrame(word_to_index.values(), columns=['index'], index=word_to_index.keys())
d.to_csv('resources/word_to_index.csv')
d = pd.DataFrame(index_to_word.values(), columns=['word'], index=index_to_word.keys())
d.to_csv('resources/index_to_word.csv')
np.save('resources/embeddings.npy', embeddings)

In [3]:
embeddings = np.load('resources/embeddings.npy')
word_to_index = pd.read_csv('resources/word_to_index.csv', index_col=0)
index_to_word = pd.read_csv('resources/index_to_word.csv', index_col=0)

In [36]:
word_to_index.loc['филе']

index    42087
Name: филе, dtype: int64

In [4]:
def word2vec(word):
    try:
        index = word_to_index.loc[word]
        return embeddings[index]
    except:
        return np.random.randn(1, 300)

In [5]:
groceries = pd.read_csv('resources/groceries_rus.csv')
groceries.columns

Index(['Date', 'Food'], dtype='object')

In [6]:
stop_words = stopwords.words('russian')

In [21]:
def merge_vecs(vecs, threshold=VEC_SIMILARITY):
    merge_happened = False
    i = 0
    while i < len(vecs) - 1:
        j = i + 1
        while j < len(vecs):
            if euclidean(vecs[i], vecs[j]) < VEC_SIMILARITY:
                vecs[i] = np.mean(vecs[[i, j]], axis=0)
                vecs = np.delete(vecs, i, axis=0)
                merge_happened = True
            j += 1
        i += 1
                
    if merge_happened:
        vecs = merge_vecs(vecs, threshold)
    return vecs

In [25]:
def arg_max_word_match(vecs, new_vec, threshold=VEC_SIMILARITY):
    if len(vecs) == 0:
        return None
    distances = [euclidean(vec, new_vec) for vec in vecs]
    sorted = np.argsort(distances)
    if distances[sorted[0]] < threshold:
        return sorted[0]

In [31]:
def vec2word(vec, threshold=VEC_SIMILARITY):
    embedding = np.fromstring(vec)
    max_match_index = arg_max_word_match(embeddings, embedding, threshold)
    word = index_to_word.iloc[max_match_index]
    return word

In [27]:
groceries_encoded = {}
for index, row in groceries.iterrows():
    date = row.Date
    items = row.Food.lower().split(', ')
    def preprocess(item):
        words = item.split(' ')
        words = [i for i in words if (i not in stop_words)]
        vecs = np.asarray([word2vec(w) for w in words])
        vec = np.mean(vecs, axis=0)
        return vec
    vecs = np.asarray([preprocess(i) for i in items])
    vecs = np.squeeze(vecs, axis=1)
    vecs = merge_vecs(vecs)
    for vec in vecs:
        groceries_vecs = [np.fromstring(k) for k in groceries_encoded.keys()]
        max_match_index = arg_max_word_match(groceries_vecs, vec)
        if max_match_index is not None:
            max_match = groceries_vecs[max_match_index]
            t = groceries_encoded[max_match.tostring()]
            new_vec = np.mean([vec, max_match], axis=0)
            del groceries_encoded[max_match.tostring()]
            groceries_encoded[new_vec.tostring()] = t + [date]
        else:
            groceries_encoded[vec.tostring()] = [date]
                

/Users/UnicornKing/venvs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  from ipykernel import kernelapp as app


In [39]:
embedding_str = list(groceries_encoded.keys())[9]
vec2word(embedding_str, 10)

/Users/UnicornKing/venvs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


None <class 'NoneType'>


TypeError: Cannot index by location index with a non-integer key

In [29]:
len(groceries_encoded)

70

In [ ]:
groceries = pd.DataFrame(columns=total_groceries)
groceries_encoded = groceries.copy()
for row in groceries.iterrows():
    for line in row.items():
        words = line.split(' ')
        embedding = np.sum([word2vec(word) for word in words])
        groceries_encoded

### Generate Training data

In [62]:
valid_length = groceries_history_binary.shape[0] - (groceries_history_binary.shape[0] % BATCH_SIZE)

Y = groceries_history_binary.values[:valid_length]
X = np.random.uniform(1, 5, (Y.shape[0],)).astype('int8')
X = np.expand_dims(X, -1)
X = np.expand_dims(X, -1)

### Build Model

In [6]:
class RecommendingModel(BaseEstimator):
    
    def __init__(self, window=3):
        self.last_tendency = None
        self.window = window
    
    def fit(self, x, y=None):
        assert x.shape[1] == 1
        self.last_tendency = np.mean(x[-self.window:] if x.shape[0] > self.window else x)
        
    def predict(self, x):
        return x > self.last_tendency
        

In [86]:
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [88]:
model.fit(X, Y, BATCH_SIZE, 1, callbacks=[checkpoint])

Epoch 1/1
   1/9728 [..............................] - ETA: 2:04 - loss: 0.0963 - acc: 0.9765

  14/9728 [..............................] - ETA: 44s - loss: 0.0892 - acc: 0.9786 

  26/9728 [..............................] - ETA: 44s - loss: 0.0892 - acc: 0.9776

  38/9728 [..............................] - ETA: 43s - loss: 0.0977 - acc: 0.9757

  50/9728 [..............................] - ETA: 42s - loss: 0.0992 - acc: 0.9756

...

9672/9728 [============================>.] - ETA: 0s - loss: 0.1009 - acc: 0.9741

9690/9728 [============================>.] - ETA: 0s - loss: 0.1009 - acc: 0.9741

9708/9728 [============================>.] - ETA: 0s - loss: 0.1009 - acc: 0.9741

9725/9728 [============================>.] - ETA: 0s - loss: 0.1009 - acc: 0.9741

9728/9728 [==============================] - 30s 3ms/step - loss: 0.1009 - acc: 0.9741


### Assessment

In [99]:
def generate_purchase_list(model, encoder, period=2):
    x = np.ones((BATCH_SIZE, 1, 1)) * period
    probs = model.predict(x)
    thresh = probs > 0.7
    thresh = np.squeeze(thresh)
    indices = np.argwhere(thresh == 1)
    decoded_groceries = encoder.inverse_transform(indices)
    return decoded_groceries

In [124]:
generate_purchase_list(model, grocery_encoder, 4)

array(['bottled beer', 'bottled water', 'brandy', 'chicken', 'chocolate',
       'coffee', 'condensed milk', 'cream cheese ', 'finished products',
       'frozen chicken', 'frozen dessert', 'frozen vegetables',
       'fruit/vegetable juice', 'hard cheese', 'hygiene articles',
       'ice cream', 'instant coffee', 'jam', 'liquor (appetizer)',
       'misc. beverages', 'onions', 'packaged fruit/vegetables', 'pasta',
       'pastry', 'pot plants', 'red/blush wine', 'root vegetables', 'rum',
       'softener', 'specialty bar', 'white bread', 'white wine',
       'whole milk', 'yogurt', 'zwieback'], dtype='<U25')